In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

df=pd.read_csv('creditcard.csv')
df.drop(columns=['Time', 'Amount'], inplace=True)

fraud_data=df[df['Class']==1]
normal_data=df[df['Class']==0].sample(n=len(fraud_data)*10,random_state=42)

selected_df=pd.concat([normal_data,fraud_data])

features=selected_df.drop("Class",axis=1)
cos_sim=cosine_similarity(features)

In [2]:
mask = cos_sim > 0.5

src, dst = np.nonzero(mask)
edge_index = np.stack([src, dst])
density = 2*src.shape[0]/(features.shape[0]*features.shape[0])
print(f'Number of edges:{len(src)}')
print(density)

Number of edges:1328646
0.09072431633407244


In [3]:
label = selected_df['Class'].values

cnt = 0
for i, j in zip(src, dst):
    if label[i] == label[j]:
        cnt += 1


print(f'homo ratio : {cnt/len(src)}')

homo ratio : 0.9644209217504136


In [4]:
X = selected_df.drop(columns=['Class']).values
y = selected_df['Class'].values

np.savez('creditcard.npz', features=X, label=y, cosine_sim=cos_sim)

In [5]:
data = np.load('creditcard.npz')

data.files

['features', 'label', 'cosine_sim']